In [1]:
import codecs
import argparse
import datetime as dt
import psycopg2
import psycopg2.extras
from sets import Set
import pandas as pd
import re

engine, _ = isir_db()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: DeprecationWarning: the sets module is deprecated


In [2]:
def get_senate(reference_number):
    matcher = re.search('([A-Z]+) ([0-9]+) INS .*', reference_number)
    if not matcher:
        return None    
    return matcher.group(1) + "-" + matcher.group(2)

In [3]:
def get_senate_num(reference_number):
    matcher = re.search('([A-Z]+) ([0-9]+) INS .*', reference_number)
    return matcher.group(2) if matcher else None

In [4]:
get_senate('KSPL 27 INS 2047 / 2014')

'KSPL-27'

# Nodes

In [5]:
creditor_df = pd.DataFrame(
    engine.execute(
        "SELECT creditor_string_id, max(creditor) as creditor_name, count(*) as c  \
         FROM v_creditors_receivables ft \
         GROUP BY creditor_string_id \
         ORDER BY c DESC \
         LIMIT 100"
    ).fetchall(), columns = ["id", "name", "count"]
).drop_duplicates().drop(labels=["count"], axis=1)
creditor_df["type"] = "creditor"
creditor_df.shape

(100, 3)

In [6]:
creditor_ids = list(creditor_df["id"].values)

In [7]:
administrators_df = pd.DataFrame(
    engine.execute(
     "SELECT at.id, at.name  \
      FROM administrators_tab at \
      JOIN insolvencies_administrators_tab iat ON iat.administrator_id=at.id \
      JOIN (SELECT it.id \
            FROM insolvency_tab it \
                  JOIN v_creditors_receivables ft2 ON it.id=ft2.insolvency_id \
            WHERE ft2.creditor_string_id is not null \
                  AND ft2.creditor_string_id = ANY(%(creditor_ids)s) \
            GROUP BY it.id) as its \
      ON iat.insolvency_id=its.id \
      GROUP BY at.id, at.name", creditor_ids=creditor_ids
    ).fetchall(), columns = ["id", "name"]
).drop_duplicates()
administrators_df["id"] = administrators_df["id"].apply(lambda id_: "adm_%d" % id_)
administrators_df["type"] = "administrator"
administrators_df.shape

(877, 3)

In [8]:
insolvencies_df = pd.DataFrame(
    engine.execute(
        "SELECT it3.id as id, it3.debtor_name as debtor_name, it3.person_type as person_type, it3.reference_number as reference_number, it3.region_id, it3.proposal_timestamp::DATE \
            FROM insolvency_tab it3  \
            JOIN (SELECT it.id \
                  FROM insolvency_tab it \
                       JOIN v_creditors_receivables ft2 ON it.id=ft2.insolvency_id \
                  WHERE ft2.creditor_string_id is not null \
                       AND ft2.creditor_string_id = ANY(%(creditor_ids)s) \
                  GROUP BY it.id) as insolvencies \
            ON it3.id=insolvencies.id", creditor_ids=creditor_ids
    ).fetchall(), columns=["id", "name", "person_type", "reference_number", "region_id", "date"]
).drop_duplicates()
insolvencies_df["type"] = "debtor"
insolvencies_df.shape

(154200, 7)

In [9]:
nodes_df = creditor_df.append(administrators_df).append(
    insolvencies_df.drop(labels=["reference_number"], axis=1)
)
nodes_df = nodes_df[["id", "name", "person_type", "region_id", "type", "date"]]
nodes_df.shape
node_ids = set(nodes_df["id"].values)

# Edges

In [10]:
insolvency_administrator_edges_df = pd.DataFrame(
    engine.execute(
        "SELECT it.id AS insolvency_id, iat.administrator_id, start_date::DATE, end_date::DATE \
         FROM insolvencies_administrators_tab iat JOIN insolvency_tab it ON iat.insolvency_id=it.id"
    ).fetchall(), columns=["insolvency_id", "administrator_id", "start_date", "end_date"]
).drop_duplicates()
insolvency_administrator_edges_df[
    "administrator_id"
] = insolvency_administrator_edges_df["administrator_id"].apply(lambda id_: "adm_%d" % id_)
insolvency_administrator_edges_df["type"] = "ins_adm"
insolvency_administrator_edges_df.shape

(191978, 5)

In [11]:
insolvency_end_dates_df = pd.DataFrame(
    engine.execute(
    "SELECT insolvency_id, MAX(state_change_timestamp)::DATE \
     FROM current_insolvency_states_tab WHERE state = 9 GROUP BY insolvency_id"
    ).fetchall(), columns=["insolvency_id", "end_date"]
)

In [12]:
insolvency_creditor_edges_df = pd.DataFrame(
    engine.execute(
        "SELECT insolvency_id, creditor_string_id, proposal_timestamp::DATE \
         FROM v_creditors_receivables \
              JOIN insolvency_tab it ON insolvency_id=it.id \
         WHERE creditor_string_id is not null"
    ).fetchall(), columns=["insolvency_id", "creditor_id", "start_date"]
).drop_duplicates()
insolvency_creditor_edges_df = insolvency_creditor_edges_df.merge(
    insolvency_end_dates_df, left_on="insolvency_id", right_on="insolvency_id", how="left"
)
insolvency_creditor_edges_df["type"] = "ins_cred"
insolvency_creditor_edges_df.shape

(1559845, 5)

In [13]:
insolvency_insolvency_edges_df = pd.DataFrame(
    engine.execute(
        "SELECT it1.id as insolvency_id1, it2.id as insolvency_id2, it1.proposal_timestamp::DATE \
         FROM insolvency_tab it1 JOIN insolvency_tab it2 ON it1.debtor_address = it2.debtor_address \
         WHERE it1.debtor_address is not null  \
               AND it2.debtor_address is not null \
               AND it1.debtor_name <> it2.debtor_name \
               AND it1.id <> it2.id"
    ).fetchall(), columns=["insolvency_id1", "insolvency_id2", "date"]
).drop_duplicates()
insolvency_insolvency_edges_df["type"] = "ins_ins"
insolvency_insolvency_edges_df.shape

(3498988, 4)

In [14]:
insolvencies_df["senate"] = insolvencies_df["reference_number"].apply(get_senate)

senate_insolvency_edges_df = insolvencies_df[
    insolvencies_df["senate"].notnull()
][["id", "senate", "date"]].rename(columns={"id": "source_id", "senate": "target_id", "date": "start_date"})
senate_insolvency_edges_df = senate_insolvency_edges_df.merge(
    insolvency_end_dates_df, left_on="source_id", right_on="insolvency_id", how="left"
).drop("insolvency_id", axis=1)
senate_insolvency_edges_df["type"] = "sen_ins"

In [15]:
edges_df = insolvency_administrator_edges_df.rename(
    columns={"insolvency_id": "source_id", "administrator_id": "target_id"}
).append(
    insolvency_creditor_edges_df.rename(
        columns={"insolvency_id": "source_id", "creditor_id": "target_id"}
    )
).append(senate_insolvency_edges_df)
# .append(
#     insolvency_insolvency_edges_df.rename(
#         columns={"insolvency_id1": "srcId", "insolvency_id2": "dstId"}
#     )
# )
edges_df.apply(lambda row: row["source_id"] in node_ids and row["target_id"] in node_ids, axis=1)
edges_df = edges_df[["source_id", "target_id", "type", "start_date", "end_date"]]
edges_df.shape

(1905981, 5)

In [16]:
assert nodes_df["id"].isnull().sum() == 0
assert edges_df["source_id"].isnull().sum() == 0
assert edges_df["target_id"].isnull().sum() == 0

In [17]:
# remove deg 0 nodes
edge_node_ids = set(edges_df["source_id"].values).union(set(edges_df["target_id"].values))
nodes_df = nodes_df[nodes_df["id"].apply(lambda id_: id_ in edge_node_ids)]
nodes_df.shape

(155177, 6)

# Saving dataset

In [18]:
nodes_df.to_csv("/home/zviri/data/isir/networks_2.0/nodes.tsv", 
                sep="\t", encoding="utf-8", index=False)

In [19]:
edges_df.to_csv("/home/zviri/data/isir/networks_2.0/edges.tsv", 
                 sep="\t", encoding="utf-8", index=False)